# Anatomy of a Simulation: A Deep Dive into the SWP Core Engine

Welcome to this deep dive into the core of the Smart Water Platform (SWP). The purpose of this notebook is to explain the fundamental components that drive any simulation within the platform. Understanding these core concepts is key to building, extending, and debugging your own water system simulations.

We will explore the key players in any simulation:
- The **`SimulationHarness`**: The central conductor of our orchestra.
- **Components**: The individual instruments, representing physical objects like reservoirs and gates.
- **Agents**: The musicians, representing autonomous entities that perceive and act.
- The **`MessageBus`**: The sheet music and cues that allow everyone to communicate.

## Section 1: The `SimulationHarness` - The Conductor

The `SimulationHarness` is the most important object for running a simulation. It acts as the central manager, responsible for:

- **Holding all the components and agents** that are part of the simulation.
- **Managing the simulation's configuration**, such as its duration (`duration`) and time step (`dt`).
- **Running the main simulation loop**, stepping through time and orchestrating the interactions between all parts.
- **Providing a `MessageBus`** for agents to communicate without being directly coupled to each other.

## Section 2: The Components - The Orchestra

Every object that changes over time in the simulation (like a reservoir, a gate, or even a controller) is a **Component**. To ensure they all work together, they must follow a contract.

This contract is defined by the `Simulatable` interface in `swp.core.interfaces`. Any `Simulatable` object must have:

- A `step(action, dt)` method: This is the core of the component. It takes an `action` (e.g., a new gate opening) and a time duration `dt`, and calculates its new state.
- A `get_state()` method: This returns a dictionary representing the component's current state (e.g., `{'volume': 1000, 'water_level': 10.5}`).
- `Parameters`: Each component is initialized with a dictionary of parameters that define its physical properties (e.g., a pipe's length and diameter).

## Section 3: The Topology - Connecting the Pieces

Water systems are networks. A reservoir flows into a pipe, which flows into a turbine. The `SimulationHarness` needs to understand this physical network, which we call the **topology**.

We build the topology by:
1. Adding each component to the harness: `harness.add_component(...)`
2. Defining the directional links between them: `harness.add_connection("upstream_component", "downstream_component")`

When `harness.build()` is called, it performs a **topological sort** on this network. This creates a specific update order to ensure that when we calculate the state of a downstream component, the state of its upstream source has already been calculated for the current time step. This avoids circular dependencies and ensures a stable simulation.

## Section 4: The `MessageBus` - The Communication Channel

In a simple simulation, the harness can directly control the components. But in a **Multi-Agent System (MAS)**, components and agents need to be decoupled. An agent controlling a gate shouldn't need a direct reference to the gate object; it should just send a message with the intent to change the gate's opening.

This is achieved with the `MessageBus`. It uses a simple **publish-subscribe** pattern:
- An agent can **publish** a message to a specific topic (e.g., `bus.publish("action.gate.opening", {'control_signal': 0.5})`).
- A component or another agent can **subscribe** to that topic (`bus.subscribe("action.gate.opening", self.handle_action_message)`). When a message is published to the topic, the subscriber's handler method is automatically called.

This keeps the system modular and allows new agents and components to be added without rewriting existing ones.

## Section 5: Case Study - A Simple MAS Control Loop

Let's put all these concepts together. The code below sets up and runs a complete multi-agent simulation. 

**Scenario**: A `Reservoir`'s water level needs to be maintained at a setpoint of 12.0m. The outflow is controlled by a `Gate`.

**The MAS**:
1. A `DigitalTwinAgent` for the reservoir perceives its state and **publishes** its `water_level` to the topic `"state.reservoir.level"`.
2. A `LocalControlAgent` **subscribes** to this topic. When it receives a new water level, its internal `PIDController` computes a new desired gate opening.
3. The `LocalControlAgent` then **publishes** this new opening to the topic `"action.gate.opening"`.
4. The `Gate` itself is **subscribed** to this action topic and adjusts its opening accordingly.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from swp.simulation_identification.physical_objects.reservoir import Reservoir
from swp.simulation_identification.physical_objects.gate import Gate
from swp.local_agents.control.pid_controller import PIDController
from swp.local_agents.control.local_control_agent import LocalControlAgent
from swp.local_agents.perception.digital_twin_agent import DigitalTwinAgent
from swp.core_engine.testing.simulation_harness import SimulationHarness

# 1. Create the Harness and get the MessageBus
simulation_config = {'duration': 300, 'dt': 1.0}
harness = SimulationHarness(config=simulation_config)
message_bus = harness.message_bus

# 2. Define communication topics
RESERVOIR_STATE_TOPIC = "state.reservoir.level"
GATE_ACTION_TOPIC = "action.gate.opening"

# 3. Create the Physical Components
gate_params = {'max_rate_of_change': 0.1, 'discharge_coefficient': 0.6, 'width': 10, 'max_opening': 1.0}
reservoir = Reservoir(name="reservoir_1", initial_state={'volume': 21e6, 'water_level': 14.0}, parameters={'surface_area': 1.5e6})
# Note: The gate is subscribed to the message bus for its actions
gate = Gate(name="gate_1", initial_state={'opening': 0.1}, parameters=gate_params, message_bus=message_bus, action_topic=GATE_ACTION_TOPIC)

# 4. Create the Agents
twin_agent = DigitalTwinAgent(agent_id="twin_agent_reservoir_1", simulated_object=reservoir, message_bus=message_bus, state_topic=RESERVOIR_STATE_TOPIC)
pid_controller = PIDController(Kp=-0.5, Ki=-0.01, Kd=-0.1, setpoint=12.0, min_output=0.0, max_output=gate_params['max_opening'])
control_agent = LocalControlAgent(agent_id="control_agent_gate_1", controller=pid_controller, message_bus=message_bus, observation_topic=RESERVOIR_STATE_TOPIC, observation_key='water_level', action_topic=GATE_ACTION_TOPIC, dt=harness.dt)

# 5. Add all components and agents to the harness
harness.add_component(reservoir)
harness.add_component(gate)
harness.add_agent(twin_agent)
harness.add_agent(control_agent)

# 6. Define the physical topology
harness.add_connection("reservoir_1", "gate_1")

# 7. Build and run the simulation
import sys
original_stdout = sys.stdout
with open('simulation_log.txt', 'w') as f:
    sys.stdout = f
    harness.build()
    # Use run_mas_simulation for agent-based simulations
    harness.run_mas_simulation()
sys.stdout = original_stdout

print("Case study simulation complete.")

## Section 6: Visualizing the Results

Now that the simulation is complete, we can use the `harness.history` (which we added in a previous step) to plot the results. The plots below show the classic behavior of a feedback control system: the controller modulates the gate opening to drive the reservoir's water level towards the desired setpoint of 12.0m.

In [ ]:
# Extract data from history
time = [h['time'] for h in harness.history]
reservoir_levels = [h['reservoir_1']['water_level'] for h in harness.history]
gate_openings = [h['gate_1']['opening'] for h in harness.history]

# Create a DataFrame for easy viewing and plotting
df = pd.DataFrame({
    'Time': time,
    'Reservoir Level': reservoir_levels,
    'Gate Opening': gate_openings
})

print("First 5 steps of the simulation history:")
print(df.head())

# Plot the results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

ax1.plot(df['Time'], df['Reservoir Level'], label='Reservoir Water Level')
ax1.axhline(y=12.0, color='r', linestyle='--', label='Setpoint (12.0m)')
ax1.set_ylabel('Water Level (m)')
ax1.set_title('MAS Feedback Control Simulation')
ax1.grid(True)
ax1.legend()

ax2.plot(df['Time'], df['Gate Opening'], label='Gate Opening', color='purple')
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Gate Opening (%)')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()

## Conclusion

You now understand the key building blocks of the Smart Water Platform. By combining a `SimulationHarness`, `Simulatable` components, a physical `topology`, and a `MessageBus` for `Agent` communication, we can create complex, decoupled, and powerful simulations of water systems.